# Exploratory Inference of LDA in Pyro 

 # Why do we care?
 We're interested in exploring probabilistic programming as it
 applies to LDA and its variants.  LDA describes an extremely
 intuitive generative process, and because of this it enjoys
 continued research into its expansions. It is also both flexible
 and interpretable.  Since everybody here knows what LDA is, we
 won't go over the details.

 But, inference is really, really hard.  Most expansions to LDA that
 get published require subtle tricks to even get inference working.

 Examples are:
   1. Supervised LDA for classification only works for real-valued supervision;
      (Blei & McAuliffe, 2008)
   2. Multiple Classification LDA was published a full year later; it required a
      subtle application of Jensen's inequality to reduce O(K^N) time to O(K^2)
      (Wang et al., 2009)
   3. Hierarchical Supervised LDA (Perotte, 2011) can't model a true 
      is-a-this-and-not-that hierarchical relationship
 Just about every incremental idea requires some special trick, no matter how
 logical the idea is.

__In spite of how understandable and flexible LDA__ is, even statisticians and
practitioners will have a tough time deploying criticizable models for their needs.

Fortunately, using Pyro to overcome intractable integrals was day 2; so let's get started
with LDA in Pyro!

In [1]:
from pres import *

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sanjeevtewani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


 # Dataset and Course of Research
 We study almost 300,000 wine reviews from WineEnthusiast.com.  This dataset is
 richly tagged, with numerical scores from 0 to 100, hierarchical region information
 (country->region->winery) and variety.
 Reviews are short but basically already bags of words; we don't anticipate many words
 being wasted on nuanced semantics or stop words, so we believe that these reviews may
 be long enough for LDA to work.
 
 > "Damp earth, black plum and dank forest herbs show
 > on the nose of this single-vineyard expression. The palate offers
 > cranberry and raspberry as well as savory soy and dried beef flavors,
 > all with earthy herbs in the background."

As such, we expect to see a progression of topic modelling capabilities as we march
down our list of models:
 - LDA,
 - LDA + classification v. supervised LDA,
 - Hierarchical LDA + classification v. supervised LDA
 - Hierarchical supervised LDA v. supervised LDA
 - Spectral Methods for supervised LDA v. supervised LDA

# Vanilla LDA
__Top Words Per Topic (10 topics)__
![](files/words_per_topic_lda.png)

There is great overlap of top words in each topic. That means our inference doesn't work. Perhaps our data is truly just a very poor fit for LDA; this was always a risk. So we tried to come up with a customised stop word list to remove words that show up in almost every review, like wine, tannin, and fruit. Then, we did different parameter initialization to tune the hyperparameters. We found random initialization to document dirichlet parameter and uniform initialization to topic dirichlet parameter yields better result. 

__LDA graphical model & mean field inference model__
![](files/lda_plate_notation.png)
To tweak the $model$ and $guide$, we compared the result for two different implementations of guide. 
1. Implement the exact same procedure as the mean field variational inference as shown in the figure above. 
2. Enumerate out the topic assignment discrete random variable $z$.

But, the same topic words overlap still exists.


# Supervised LDA?

If the objective of LDA may itself not offer enough rewards to separate out topics, an extra loss may help; we can use sLDA to potentially learn more latent structure.

Supervised LDA (sLDA) is a variant of LDA by adding an additional gaussian regression on topic assignment latent variable z and using the average topic assignments of words in a review to predict the score of the review.

__sLDA graphical model__
![](files/sLDA_graphical_model.png)
Thus, in inference, to estimate the $\eta$ and $\sigma$, we put normal prior on them and let Pyro to derive an approximate posterior.


# sLDA's top words per topic
![](files/words_per_topic_slda.png)

We still see word overlap showing up. At this point, we have an inkling that our data simply won't permit factorizing via LDA. But, convinced by the richness of the dataset and the natural topic-variety mapping in our heads, we remain undeterred.  Rather, we conclude that we can check our work by fitting the same topic model via CAVI, using code widely available on the internet (from blei-lab).


# ALL VISUALIZATIONS

## Data distribution

__peek response variable distribution(scaled)__
![](files/score_historgram.png)
After scaling 

## LDA result

## sLDA result

### 


In [2]:
plot_pyro_lda_1_theta_tsne()

Loading BokehJS ...

# Posterior Collapse

We suspect that what we're experiencing is a collapse in all of our estimated distributions to the same point.  This could be MAP or another local optimum, and the literature seems vaguely aware that this can happen for topic models:

>"...despite some notable successes ...,black box inference methods are significantly more challenging to apply to topic models. For example, in initial experiments, we tried to apply ADVI (Kucukelbir et al., 2016), ..., but it was difficult to obtain any meaningful topics. Two main challenges are: first, the Dirichlet prior ... hinders reparameterisation, and second, the well known problem of component collapsing (Dinh & Dumoulin, 2016), in which the inference network becomes stuck in a bad local optimum in which all topics are identical" (Srivastava and Sutton, 2017)

However, most references to the phenomenon are in relation to VAEs, where it is also referred to as KL or mode collapse.  The cause is not well understood (Lucas et al., 2019), but the accepted definition is that it occurs when the distribution of latent variable $z$ given variational parameter $\theta$ is fit such that $q(z \vert \theta) = p(z)$; samples of the latent variable become independent of the variational parameters.

We hope that by scrutinizing each parameter update from Pyro and CAVI, we can get a sense of what's going on.  We can either trace parameters in Pyro (which is difficult to do because of how many parameters there are for LDA), or we can hand-derive the parameter updates it uses.  Pyro employs BBVI for discrete distributions; it will furthermore use context clues from plates to Rao-Blackwellize where possible, so for the remainder of this discussion we will work with the analytical updates we believe Pyro is running.

# Parameter Updates

Recall the parameter updates from CAVI:
![](files/cavi_updates.png)


Versus the same updates for BBVI:
\begin{align}
&\gamma \leftarrow \gamma + \rho\nabla_\gamma L(\gamma); \qquad \phi_{d,n} \leftarrow \phi_{d,n} + \rho\nabla_{\phi_{d,n}} L; \\
&\nabla_\gamma L = E_{q(\theta_d \vert \gamma)}\left[\nabla_\gamma \log q(\theta_d \vert \gamma) \left( \log p (\theta_d \vert \alpha) - \log q(\theta_d \vert \gamma) + E_{q(z_d \vert \phi_d)}[\log p(z_d \vert \theta_d)] \right) \right] \\
&\nabla_{\phi_{d,n}} L \\
&\:\:\:= E_{q(z_{d,n} \vert \phi_{d,n})}[\nabla_{\phi_{d,n}} \log q(z_{d,n} \vert \phi_{d,n}) \\
&\qquad\times \left( \log p (w_{d,n} \vert z_{d,n}, \beta) - \log q (z_{d,n} \vert \phi_{d,n}) + E_{q(\theta_d \vert \gamma)}[\log p(z_{d,n} \vert \theta_d)] \right) ] \\
\end{align}


In practice, we estimate these expectations Via Monte-Carlo, which we can write down (over $B$ samples)
\begin{align}
\nabla&_\gamma L(\gamma) \approx \frac{1}{B} \sum_{d=1}^B \left(\log \theta_d - \Psi(\gamma) + \Psi\left(\sum_{i=1}^K \gamma _i\right)\right) \\
& \qquad\times \left(\log(Dir(\theta_d \vert \alpha))-\log(Dir(\theta_d \vert \gamma)) + \frac{1}{N_d} \sum_{n=1}^{N_d} \log \theta_{d}^T \bar{z}_{d,n} \right)
\end{align}

\begin{align}
\nabla&_{\phi_d,n} L(\phi_{d, n}) \approx \frac{1}{N_d}\sum_{n=1}^{N_d} \frac{1}{\phi_{d,n,z_{d,n}}} \left(\log \beta_{z_{d,n},w_{d,n}} - \log \phi_{d,n,z_{d,n}} + \frac{1}{B} \sum_{d=1}^B \log \theta_d^T z_{d,n} \right)
\end{align}

\begin{align}
\nabla&_{\lambda_k} L(\lambda_k)\\
&\approx \frac{1}{B} \sum_{d=1}^B \left(\log \beta_k - \Psi(\lambda) + \Psi\left( \sum_{v=1}^V \lambda_v\right)\right) \\
&  \qquad \times \left(\frac{1}{N_d} \sum_{n=1}^{N_d} \log \beta_{k}^T w_{d, n} 1\{z_{d,n} = k\} + \log Dir (\beta_k \vert \eta_k) - \log Dir (\beta_k \vert \lambda_k) \right)
\end{align}

A few things to note:
 - Updates aren't comparable; CAVI is run until convergence while BBVI will step iteratively (along with other parameters).  But the similarities are evident
 - All updates show some tradeoff between updating a parameter and shrinking toward its prior. $\theta$ isn't the prior of $\phi$, but they are closely related via the model.
 - Both BBVI updates have Monte-Carlo estimates that can be written as KL divergences

How so?

![](files/bbvi_updates_as_kls.png)


We expanded the second expectation by writing $p(z_{d,n}\vert\theta_d) = p(\theta_d\vert z_{d,n}) p(z_{d,n}) / p(\theta_d)$.  There are two things to note:
 - The second KL divergence in the gradient for $\phi$ can be ignored, because $\theta_d$ and $\gamma$ aren't in the Markov blanket of $\phi_{d,n}$
 - $\log(p(z_{d,n}) / q(z_{d,n} \vert \phi_{d,n}))$ should trigger some alarm bells.  This term will guide our variational distribution towards the unconditional $p(z)$: the hallmark of posterior collapse.  If these variational parameters are guided towards a collapsed maximum, the iterative gradient updates will drag all model parameters towards the collapse.
 
__We believe that the noisy gradient updates in BBVI__ force us towards posterior collapse.  In the Monte Carlo expressions for the expectations above, you can see that __unlike with CAVI, at each iteration other variational parameters will only update the current one through dot products and KL divergences__.  Importantly, if the projection of our parameter on the others is small, all of the linkages in the model break down, and we experience a collapse.  Potentially we can overcome this with larger subsamples (reducing noise), but even Rao-Blackwellized estimation is too noisy for the sheer number of parameters we have.

Note that we cannot work with either of $p(z)$ or $p(\theta\vert z)$; at best we can posit that the tradeoff in the updates for $\phi$ can lead to poor local optimums.

# Conclusions

### Summary

We explored LDA and its variant sLDA in probabilistic programming language and tried to do inference on real world wine review text data. To dive deeper into why Pyro is not working, we tested our data with hand written CAVI and analyzed the gradient procedures in BBVI. We concluded that BBVI fails because of the big variance in each update as well as the simultaneous update of topic-word distributions and document-topic distributions. 

Models variation: LDA (with different versions of pyro implementation), sLDA
Inference techniques: Variational Inference (CAVI and BBVI)
Implementation strategies: Pyro

### Challenges

1. Getting Pyro-implemented LDA inference to work. We tried different implementation tricks, stop words removal and hypyterparameter tuning.
2. Analyzing why Pyro fails in this BBVI procedure. We hand derived BBVI for Pyro and observe step by step how parameters are updated.

### Future Work

Identifying the problem in BBVI informs our next steps.  Options are:

1. Set appropriate subsample size at each step; in experiments, we see we need very large batch sizes, (whereas batch VI succeeds with very small batches) (no good)
2. Consider models without $\phi$.  Given that we don't need to worry about tractability, perhaps we can use a collapsed LDA model.  (no good)
3. Try models that work with different priors, like logistc normals.



 # CAVI for LDA

 # References
 (Blei & McAuliffe, 2008) https://papers.nips.cc/paper/3328-supervised-topic-models.pdf
 (Wang et al., 2009) http://vision.stanford.edu/pdf/WangBleiFei-Fei_CVPR2009.pdf
 (Schroeder, 2018) https://edoc.hu-berlin.de/bitstream/handle/18452/19516/thesis_schroeder_ken.pdf?sequence=3
 (Perotte et al., 2011) https://papers.nips.cc/paper/4313-hierarchically-supervised-latent-dirichlet-allocation
 (Thoutt, 2017) https://www.kaggle.com/zynicide/wine-reviews
 (Srivastava and Sutton, 2017) https://arxiv.org/pdf/1703.01488.pdf
 (Lucas et al., 2019) https://openreview.net/pdf?id=r1xaVLUYuE